In [1]:
# se cargan todas las librerias
# Este notebook se corrio en python 3.9.13
from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np
import pandas as pd 
import json
import spacy
import re
import unicodedata
import inflect
import keras 
import tensorflow 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences 
#from tensorflow.keras.preprocessing.text import Tokenizer
#from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np 
from datasets import Dataset, Features, ClassLabel, Value, DatasetDict, Sequence 
from sklearn.preprocessing import MinMaxScaler 
import torch.nn as nn
from torch.nn import Linear  

d:\reto_ia\ia\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# se cargan los datos
train = pd.read_csv('train.csv')
test_q = pd.read_csv('test_queries.csv')
test_d = pd.read_csv('test_documents.csv')


# Se crean diferentes funciones para la limpieza de texto.
def quitar_tildes(texto):
    """
    Elimina los tildes de una cadena de texto.

    Parámetros:
    - texto: Cadena de texto con tildes.

    Retorna:
    Una nueva cadena de texto sin tildes.
    """
    texto_sin_tildes = unidecode(texto)
    return texto_sin_tildes


# Cargar el modelo de Spacy para inglés
nlp = spacy.load("es_core_news_sm")

# Funcion para quitar caracteres especiales de un texto utilizando expresiones regulares
def remove_special_characters(text):
    # Aplicar el modelo de Spacy para dividir el texto en tokens
    doc = nlp(text)

    # Reconstruir el texto eliminando caracteres especiales
    cleaned_text = ' '.join([token.text for token in doc if not re.search('[^A-Za-z0-9]+', token.text)])

    return cleaned_text

def remove_hashtag_and_url(words):
    # Eliminar URLs
    words = re.sub(r'http\S+|www.\S+', '', words)
    # Eliminar hashtags
    words = re.sub(r'#\S+', '', words)
    # Eliminar usuarios @
    words = re.sub(r'@\S+', '', words)
    # Elimina palabras con mas de aa
    words =re.sub(r'\b\w*aa\w*\b','',words)
    # Elimina palabras con mas de hh
    words =re.sub(r'\b\w*hh\w*\b','',words)
    # Elimina palabras con mas un ja r'mm+'
    words =re.sub(r'(ja){2,}','',words)
    # Elimina palabras con mas un mm
    words =re.sub(r'mm+','',words)

    return words



def remove_non_ascii(words):
    """
    Esta función toma una lista de palabras y elimina los caracteres no ASCII de cada palabra.
    
    Parámetros:
    - words: Una lista de palabras que se van a limpiar.
    
    Retorna:
    - Una cadena de texto que contiene las palabras limpias, sin caracteres no ASCII.
    """
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_word = new_word.replace('_', '')
        new_word = new_word.replace('\n', '')
        new_words.append(new_word)
    return str.join('', new_words)


def reduce_repeated_characters(word, max_repeats=2):
    # Utilizar una expresión regular para encontrar repeticiones excesivas
    pattern = re.compile(r'(\w)\1{%d,}' % (max_repeats - 1))

    # Función de sustitución para reducir repeticiones
    def reduce_match(match):
        return match.group(1) * max_repeats

    # Aplicar la función de sustitución a la palabra
    reduced_word = pattern.sub(reduce_match, word)

    return reduced_word


def to_lowercase(words):
    #Convert all characters to lowercase from list of tokenized words
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return str.join('',new_words)

def remove_punctuation(text):
    # Remove punctuation from text using regular expression
    text_without_punctuation = re.sub(r'[^\w\s]', '', text)
    return text_without_punctuation


# Inicializar el objeto de la clase Inflect
p = inflect.engine()

# Función para quitar numeros

def remove_numbers(text):
    words = []
    for word in text.split():
        # Verificar si la palabra contiene solo dígitos
        if not re.match(r'^\d+$', word):
            words.append(word)
    return ' '.join(words)


def remove_stopwords_spacy(text):
    """
    Stopwords de la libreria de spacy

    """
    doc = nlp(text)
    palabras_sin_stopwords = [token.text for token in doc if not token.is_stop]
    return ' '.join(palabras_sin_stopwords)

mis_stopwords = {"a", "al", "algo", "algunas", "algunos", "ante", "antes", "como", "con", "contra",
        "cual", "cualquier", "cualquieras", "cuales", "cualesquiera", "cuando", "de", "deja",
        "dejais", "dejamos", "dejan", "dejar", "del", "demas", "demasiada", "demasiadas",
        "demasiado", "demasiados", "dentro", "donde", "durante", "e", "el", "ella", "ellas",
        "ellos", "en", "entre", "era", "erais", "eramos", "eran", "eras", "eres", "es", "esa",
        "esas", "ese", "eso", "esos", "esta", "estaba", "estabais", "estábamos", "estaban",
        "estabas", "estad", "estada", "estadas", "estado", "estados", "estamos", "estando",
        "estar", "estara", "estaran", "estaras", "estare", "estareis", "estaremos", "estaría",
        "estaríais", "estaríamos", "estarian", "estarias", "estas", "este", "esto", "estos",
        "estoy", "estuve", "estuviera", "estuvierais", "estuvieran", "estuvieras", "estuvieron",
        "estuviese", "estuvieseis", "estuviesemos", "estuviesen", "estuvieses", "estuvimos",
        "estuviste", "estuvisteis", "estuvo", "está", "estabamos", "estais", "están", "estás",
        "fue", "fuera", "fuerais", "fueran", "fueras", "fueron", "fuese", "fueseis", "fuésemos",
        "fuesen", "fueses", "fui", "fuimos", "fuiste", "fuisteis", "ha", "habida", "habidas",
        "habido", "habidos", "habiendo", "habremos", "habrá", "habrán", "habrás", "habré",
        "habreis", "habremos", "habría", "habríais", "habríamos", "habrian", "habrias", "habeis",
        "había", "habíais", "habíamos", "habían", "habías", "han", "has", "hasta", "hay", "haya",
        "hayamos", "hayan", "hayas", "hayáis", "he", "hemos", "hube", "hubiera", "hubierais",
        "hubieran", "hubieras", "hubieron", "hubiese", "hubieseis", "hubiesemos", "hubiesen",
        "hubieses", "hubimos", "hubiste", "hubisteis", "hubo", "la", "las", "le", "les", "lo", "los",
        "me", "mi", "mis", "mucho", "muchos", "muy", "mas", "mi", "mia", "mias", "mio", "mios", "nada",
        "ni", "no", "nos", "nosotras", "nosotros", "nuestra", "nuestras", "nuestro", "nuestros", "o",
        "os", "otra", "otras", "otro", "otros", "para", "pero", "poco", "por", "porque", "que",
        "quien", "quienes", "qué", "se", "sea", "seamos", "sean", "seas", "sentid", "sentida",
        "sentidas", "sentido", "sentidos", "seremos", "sera", "seran", "seras", "seré", "seréis",
        "seremos", "seria", "seriais", "seríamos", "serían", "serías", "seáis", "siente", "sin",
        "sintiendo", "sobre", "sois", "somos", "son", "soy", "su", "sus", "suya", "suyas", "suyo",
        "suyos", "si", "tambien", "tanto", "te", "tendremos", "tendra", "tendran", "tendras", "tendre",
        "tendréis", "tendríamos", "tendría", "tendríais", "tendríamos", "tendrían", "tendrías",
        "tened", "tenemos", "tenga", "tengamos", "tengan", "tengas", "tengo", "tengais", "tenida",
        "tenidas", "tenido", "tenidos", "teniendo", "teneis", "tenia", "teniais", "teniamos", "tenian",
        "tenias", "ti", "tiene", "tienen", "tienes", "todo", "todos", "tu", "tus", "tuve", "tuviera",
        "tuvierais", "tuvieran", "tuvieras", "tuvieron", "tuviese", "tuvieseis", "tuviésemos", "tuviesen",
        "tuvieses", "tuvimos", "tuviste", "tuvisteis", "tuvo", "tuya", "tuyas", "tuyo", "tuyos", "tú",
        "un", "una", "uno", "unos", "vosotras", "vosotros", "vuestra", "vuestras", "vuestro", "vuestros",
        "y", "ya", "yo", "él", "eramos"}

# Función para quitar stopwords con spaCy, incluyendo las stopwords personalizadas
def remove_stopwords_spacy(text):
    for palabra in mis_stopwords:
        nlp.Defaults.stop_words.add(palabra)

    doc = nlp(text)
    palabras_sin_stopwords = [token.text for token in doc if not token.is_stop]

    # Verificar si la lista de palabras sin stopwords está vacía
    if palabras_sin_stopwords:
        return ' '.join(palabras_sin_stopwords)
    else:
        return text


def lemmatize_verbs(text):
    lemmatized_tokens = []
    doc = nlp(text)
    for token in doc:
        try:
            lemma = token.lemma_
            if lemma:
                lemmatized_tokens.append(lemma)
            else:
                lemmatized_tokens.append(token.text)  # Agregar la palabra original si no hay lema
        except Exception as e:
            print(f"Error al lematizar el token '{token}': {e}")
            lemmatized_tokens.append(token.text)  # Agregar la palabra original en caso de error

    lemmatized_text = ' '.join(lemmatized_tokens)
    return lemmatized_text 

def normalizador(df,columna):
    df['text2'] = df[columna].apply(quitar_tildes)
    df['text2'] = df['text2'].apply(remove_punctuation)
    df['text2'] = df['text2'].apply(remove_special_characters)
    df['text2'] = df['text2'].apply(remove_non_ascii)
    df['text2'] = df['text2'].apply(reduce_repeated_characters)
    df['text2'] = df['text2'].apply(to_lowercase)
    df['text2'] = df['text2'].apply(remove_numbers)
    df['text2'] = df['text2'].apply(remove_stopwords_spacy)
    df['text2'] = df['text2'].apply(lemmatize_verbs)
    return df['text2']


#train_document = normalizador(train, 'document')
#train_query =  normalizador(train, 'query')

#test_document = normalizador(test_d, 'document')
#test_query  = normalizador(test_q, 'query')

train_document = pd.read_csv('train_document_vector.csv')
train_query = pd.read_csv('train_query_vector.csv')
train_label = pd.read_csv('train_label_vector.csv')

test_document = pd.read_csv('test_document_vector.csv')
test_query = pd.read_csv('test_query_vector.csv')

from torch.utils.data import DataLoader, Dataset
from transformers import AutoModel, AutoTokenizer, AdamW
from torch.nn import BCEWithLogitsLoss

# Define el nombre del modelo y carga el modelo y el tokenizador
model_name = "sentence-transformers/paraphrase-MiniLM-L6-v2"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_and_pad(df, text_column, tokenizer):
    sequences = tokenizer(df[text_column].tolist(), return_tensors='pt', padding=True, truncation=True, max_length = 120)
    return sequences['input_ids'], sequences['attention_mask'], sequences['token_type_ids']

input_train_queries, attention_mask_train_queries, token_type_ids_train_queries = tokenize_and_pad(train_query, 'text2', tokenizer=tokenizer)
input_train_documents, attention_mask_train_documents, token_type_ids_train_documents = tokenize_and_pad(train_document, 'text2', tokenizer=tokenizer)

input_test_queries, attention_mask_test_queries, token_type_ids_test_queries  = tokenize_and_pad(test_query, 'text2', tokenizer=tokenizer)
input_test_documents, attention_mask_test_documents, token_type_ids_test_documents = tokenize_and_pad(test_document, 'text2', tokenizer=tokenizer)



In [3]:
new_output_shape = 4000

# Tamaño actual de la salida del modelo
current_output_shape = model.pooler.dense.out_features

# Verificar las diferencias en las dimensiones
diff_shape = new_output_shape - model.pooler.dense.out_features

# Agregar una capa lineal para ajustar las dimensiones
model.pooler.dense = nn.Linear(384,current_output_shape + diff_shape)

# Imprimir el modelo modificado
print(model)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 384, padding_idx=0)
    (position_embeddings): Embedding(512, 384)
    (token_type_embeddings): Embedding(2, 384)
    (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=384, out_features=384, bias=True)
            (key): Linear(in_features=384, out_features=384, bias=True)
            (value): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=384, out_features=384, bias=True)
            (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [4]:
import torch.nn as nn
from torch.nn import Linear 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Se crea el diccionario con las columnas input, attention_mask y token_type_ids para poder ingresarlos al modelo de hugging face.

data_train_queries_dict = {
    'input_ids': input_train_queries,
    'attention_mask': attention_mask_train_queries,
    'token_type_ids' : token_type_ids_train_queries
}

data_train_documents_dict = {
    'input_ids': input_train_documents,
    'attention_mask': attention_mask_train_documents,
    'token_type_ids': token_type_ids_train_documents
}


model 
optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss() 

for epoch in range(3):
    model.train()
    optimizer.zero_grad()

    with torch.set_grad_enabled(True): # calcular los gradiantes
        queries_tokenized = data_train_documents_dict 
        documents_tokenized = data_train_queries_dict

        queries_embeddings = model(**queries_tokenized)
        documents_embeddings = model(**documents_tokenized)

        queries_array =  queries_embeddings.last_hidden_state.view(4000, -1)
        documents_array = documents_embeddings.last_hidden_state.view(4000, -1)


        # Se verifica si queries_array es la misma dimension que documents_array 
        tensor1 = queries_array
        tensor2 = documents_array 
        if tensor2.size() != tensor1.size():
            # Calcular la diferencia en dimensiones
            dif_dim = tensor1.size(1) - tensor2.size(1)
            
            # Verificar si tensor2 es menor que tensor1
            if dif_dim > 0:
                # Agregar ceros al final para hacer que tensor2 tenga las mismas dimensiones que tensor1
                zeros_to_add = torch.zeros(tensor2.size(0), dif_dim)
                tensor2 = torch.cat((tensor2, zeros_to_add), dim=1)
            else:
                print("El tensor 2 no es compatible con el tensor 1.")
        else:
            print("Los tensores tienen las mismas dimensiones.")

        # Verificar las nuevas dimensiones de tensor2
        print("Dimensiones de tensor2 después de la modificación:", tensor2.size()) 
        # Realizar el calculo de la similitud
        outputs = torch.cosine_similarity(tensor1, tensor2)

        loss = criterion(outputs, torch.tensor(train['label']).float()) 

        # Realizar la retropropagación y la optimización
        loss.backward()
        optimizer.step()

# Guardar tu modelo entrenado, si lo deseas
torch.save(model.state_dict(), 'modelo_busqueda.pth')

d:\reto_ia\ia\lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Dimensiones de tensor2 después de la modificación: torch.Size([4000, 46080])
Dimensiones de tensor2 después de la modificación: torch.Size([4000, 46080])
Dimensiones de tensor2 después de la modificación: torch.Size([4000, 46080])


In [8]:
print('La funcion de perdida es: ', loss)  

La funcion de perdida es:  tensor(3308.7229, grad_fn=<DivBackward1>)


# testeo 

In [4]:
# Se define una funcion para obtener el tensor del texto y su codificacion al ser pasado al tokenizer

def tokenize_and_pad_2(df, text_column, tokenizer):
    # Tokenizar y generar secuencias
    sequences = tokenizer(df[text_column].tolist(), return_tensors='pt', padding=True, truncation=True, max_length=120)
    
    # Obtener los tokens de entrada, máscara de atención y tipo de tokens
    input_ids = sequences['input_ids']
    attention_mask = sequences['attention_mask']
    token_type_ids = sequences['token_type_ids']
    
    # Decodificar los tokens para obtener las palabras originales
    decoded_texts = [tokenizer.decode(ids, skip_special_tokens=True) for ids in input_ids]

    # Almacenar el tensor asociado y la palabra asociada a ese tensor
    token_tensor_pairs = []
    for ids, decoded_text in zip(input_ids, decoded_texts):
        token_tensor_pairs.append((ids, decoded_text))

    tensor_list, word_list = zip(*token_tensor_pairs) 
    # Convertir las listas a tensores
    token_tensors = torch.stack(tensor_list)
    words = list(word_list)

    return  token_tensors, words 


tensor_queries_tensor, tensor_queries_word = tokenize_and_pad_2(test_query, 'text2', tokenizer=tokenizer)
tensor_documemt_tensor, tensor_document_word = tokenize_and_pad_2(test_document, 'text2', tokenizer=tokenizer)

In [5]:
# Se crean los diccionarios con las columnas input, attention_mask y token_type_id
data_test_queries_dict = {
    'input_ids': input_test_queries,
    'attention_mask': attention_mask_test_queries,
    'token_type_ids' : token_type_ids_test_queries
    # 'word': tensor_queries_word
}

data_test_documents_dict = {
    'input_ids': input_test_documents,
    'attention_mask': attention_mask_test_documents,
    'token_type_ids': token_type_ids_test_documents
    #'word': tensor_document_word
}

In [6]:
# Se carga el modelo guardado anteriormente y se pone en modo evaluacion
model.load_state_dict(torch.load('modelo_busqueda.pth'))
model.eval()  # Coloca el modelo en modo de evaluación

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 384, padding_idx=0)
    (position_embeddings): Embedding(512, 384)
    (token_type_embeddings): Embedding(2, 384)
    (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=384, out_features=384, bias=True)
            (key): Linear(in_features=384, out_features=384, bias=True)
            (value): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=384, out_features=384, bias=True)
            (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [8]:
# Estae codigo deja en las dimensiones del tensor1_ y le reduce la dimensionalidad al tensor2_ para poder calcular la similitud
from sentence_transformers import SentenceTransformer, util


# Pasar los datos de prueba a través del modelo cargado
with torch.no_grad():  
    # Deshabilitar el cálculo de gradientes durante la inferencia
    test_documents_embedding = model(**data_test_documents_dict).last_hidden_state.view(100, -1)
    test_queries_embedding = model(**data_test_queries_dict).last_hidden_state.view(100, -1)

    tensor1_ = test_queries_embedding 
    tensor2_ = test_documents_embedding 

    if tensor2_.size() != tensor1_.size():
        # Calcular la diferencia en dimensiones
        dif_dim_ = abs(tensor1_.size(1) - tensor2_.size(1))
        
        # Verificar cuál tensor es mayor
        if tensor2_.size(1) > tensor1_.size(1):
            # Reducir la dimensión de tensor2_ para que tenga las mismas dimensiones que tensor1_
            tensor2_ = tensor2_[:, :tensor1_.size(1)]
        else:
            # Agregar ceros al tensor 2 para hacer que tenga las mismas dimensiones que tensor 1
            zeros_to_add_ = torch.zeros(tensor2_.size(0), dif_dim_)
            tensor2_ = torch.cat((tensor2_, zeros_to_add_), dim=1)
    else:
        print("Los tensores tienen las mismas dimensiones.")

    # Verificar las nuevas dimensiones de tensor2_
    print("Dimensiones de tensor2_ después de la modificación:", tensor2_.size()) 
    
    consulta_y_documentos_similares_id = []
    consulta_y_documentos_similares = []

    for i in range(tensor1_.size(0)):
    # Calcular la similitud coseno entre la consulta y cada documento
        similarities_ = util.cos_sim(tensor1_[i], tensor2_)
        word_query = tensor_queries_word[i]
        word_query_id = test_q['query_id'][i]
        # Obtener los índices de los documentos más similares (en orden descendente)
        top_document_indices_ = similarities_.argsort(descending=True)[0][:10] 

        documentos_similares_id = []
        documentos_similares = []
        for index in range(10):
            indice = top_document_indices_.tolist()[index]
            documento_similar_id = test_d['document_id'][indice] 
            documento_similar = tensor_document_word[indice]
            documentos_similares_id.append(documento_similar_id) 
            documentos_similares.append(documento_similar) 


        consulta_y_documentos_similares_id.append({'query_id': word_query_id, 'document_id': documentos_similares_id})
        consulta_y_documentos_similares.append({'query': word_query, 'document_id': documentos_similares})


Dimensiones de tensor2_ después de la modificación: torch.Size([100, 35712])


In [9]:
# Crear un diccionario para almacenar los resultados
results_dict_id = {}

# Procesar cada consulta y encontrar los documentos más similares
for i in range(len(consulta_y_documentos_similares_id)):
    # Obtener el word_query_id correspondiente a esta consulta
    word_query_id = consulta_y_documentos_similares_id[i]['query_id']
    # Agregar los IDs de los documentos al diccionario
    results_dict_id[word_query_id] = consulta_y_documentos_similares_id[i]['document_id']

# Convertir el diccionario a formato JSON
import json
json_results_id = json.dumps(results_dict_id, indent=4)

# Imprimir los resultados JSON
print(json_results_id)

{
    "KWjLrafk": [
        "6ZTWDGPA",
        "53Q5nZCn",
        "t7MFggR0",
        "UphLchV1",
        "34pM6WBS",
        "kPEDk3X6",
        "VZ8Ui042",
        "gD53zFDC",
        "pCp2q04C",
        "olvCZ9Iy"
    ],
    "S26n47qu": [
        "6ZTWDGPA",
        "53Q5nZCn",
        "t7MFggR0",
        "UphLchV1",
        "34pM6WBS",
        "kPEDk3X6",
        "gD53zFDC",
        "olvCZ9Iy",
        "EfHVfhi9",
        "pCp2q04C"
    ],
    "Jj7Rmaeu": [
        "6ZTWDGPA",
        "53Q5nZCn",
        "t7MFggR0",
        "UphLchV1",
        "34pM6WBS",
        "kPEDk3X6",
        "gD53zFDC",
        "pCp2q04C",
        "olvCZ9Iy",
        "VZ8Ui042"
    ],
    "XxxdpKqh": [
        "6ZTWDGPA",
        "t7MFggR0",
        "UphLchV1",
        "53Q5nZCn",
        "kPEDk3X6",
        "34pM6WBS",
        "gD53zFDC",
        "VZ8Ui042",
        "olvCZ9Iy",
        "pCp2q04C"
    ],
    "Ix7Dq5ey": [
        "6ZTWDGPA",
        "t7MFggR0",
        "53Q5nZCn",
        "UphLchV1",
    

In [10]:
# guardamos los resultados
ruta_archivo = 'resultado_semantic_search.json'

# Guarda los datos en el archivo JSON
with open(ruta_archivo, 'w') as archivo:
    json.dump(results_dict_id , archivo) 